In [27]:
from time import sleep
from random import random
import pandas as pd
import requests
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib.parse as urlparse
from urllib.parse import parse_qs

# Constants
BASE_URL = 'https://www.flipkart.com/'
SEARCH_QUERY = "phones"
TOP_N_PRODUCTS = 2
REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT = 2 #10 Reviews exist per page

SAMPLE_URL = "https://www.flipkart.com/poco-m6-pro-5g-power-black-128-gb/p/itmef8fa46f89738?pid=MOBGRNZ3ER4N3K4F&lid=LSTMOBGRNZ3ER4N3K4FIYYGCU&marketplace=FLIPKART&q=phones&store=tyy%2F4io&spotlightTagId=TrendingId_tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=Search&iid=4b216b86-7d32-47a6-924d-a4b1f3d52950.MOBGRNZ3ER4N3K4F.SEARCH&ppt=sp&ppn=sp&ssid=ygv5uzlqb40000001693306294200&qH=28388ea49f54c5b8"
r = requests.get(SAMPLE_URL)    
soup = BeautifulSoup(r.content, 'html.parser') 
print(soup.prettify()[:500])

# Extracting all review blocks
## Note col._2wzgFH.K0kLPL means 3 entities namely 'col', ' _2wzgFH' and 'K0kLPL' 
## This is written in HTML as 'col _2wzgFH K0kLPL'
## This can also be seen in Bullet 3

rows = soup.find_all('div',attrs={'class':'col _2wzgFH'})
print(f"Count of rows(reviews):{len(rows)}\n\n\n")
# iteration over all blocks
for row in rows:
    # Print a sample row(review html block)
    # print(f"row:\n{row} \n\n")
    
    # finding all rows within the block
    sub_row = row.find_all('div',attrs={'class':'row'})
        
    # extracting text from 1st and 2nd row
    rating = sub_row[0].find('div').text
    print(f"rating:{rating} \n\n")
    
    summary = sub_row[0].find('p').text
    print(f"summary:{summary} \n\n")
    
    review = sub_row[1].find_all('div')[2].text
    print(f"review:{review} \n\n")
    
    location = sub_row[3].find('p',attrs={'class':'_2mcZGG'}).find_all('span')[1].text
    location = "".join(location.split(",")[1:]).strip()
    print(f"location:{location} \n\n")
    
    date = sub_row[3].find_all('p',attrs={'class':'_2sc7ZR'})[1].text
    print(f"date:{date} \n\n")
    
    
    sub_row_2 = row.find_all('div',attrs={'class':'_1e9_Zu'})[0].find_all('span',attrs={'class':'_3c3Px5'})
    
    upvotes = sub_row_2[0].text
    print(f"upvotes:{upvotes} \n\n")
    
    downvotes = sub_row_2[1].text
    print(f"downvotes:{downvotes} \n\n")
    
    break  
  
def get_popular_product_s_titles_and_urls(search_query : str, popular_products_count_limit : int = None):
    
    search_url = f"{BASE_URL}search?q={SEARCH_QUERY}&sort=popularity"
    search_response = requests.get(search_url)
    
    # Pause the loop for 1-3 seconds to simulate natural setting not overwhelm the server with back to back requests without any pause
    # sleep(randint(1,3))
    
    search_html_soup = BeautifulSoup(search_response.content, 'html.parser')
    search_results_products = search_html_soup.find_all('div',attrs={'class':'_2kHMtA'})
    
    product_titles, product_urls,retail_prices,discount_perc,discounted_price = [],[],[],[],[]
    
    product_count = 0
    
    
    for product in tqdm(search_results_products, desc="Search Results Iteration", position=0, leave=True):
        
        ad_mention_subrow = product.find("div", attrs={"class":"_4HTuuX"})
        
        is_ad = not not ad_mention_subrow
        
        if not is_ad:
            
            title_mention_subrow = product.find("a", attrs={"class":"_1fQZEK"})
            product_title=product.find('div', attrs={"class":"_4rR01T"}).text
            product_relative_url = title_mention_subrow["href"]
            Retail_Price= product.find('div', attrs={"class":"_3I9_wc _27UcVY"}).text
            DisPer=product.find('div', attrs={"class":"_3Ay6Sb"}).text
            Disc_price=product.find('div', attrs={"class":"_30jeq3 _1_WHN1"}).text
            #print(product_relative_url)
            product_url = urljoin(BASE_URL,product_relative_url)
            
            parsed_url = urlparse.urlparse(product_url)
            parsed_url_path = parsed_url.path
            parsed_url_path_split = parsed_url_path.split("/")
            parsed_url_path_split[2] = "product-reviews"
            parsed_url_path_modified = "/".join(parsed_url_path_split)
            parsed_url_modified = parsed_url._replace(path=parsed_url_path_modified)
            product_url = parsed_url_modified.geturl()
            
            
            product_titles.append(product_title)
           
            product_urls.append(product_url)
            retail_prices.append(Retail_Price)
            discount_perc.append(DisPer)
            discounted_price.append(Disc_price)
            product_count += 1
            
            if popular_products_count_limit and (product_count >= popular_products_count_limit):
                break
    print(product_titles) 
    print(product_urls)
    return product_titles, product_urls,retail_prices,discount_perc,discounted_price
    
product_titles, product_urls,retail_prices,discount_perc,discounted_price = get_popular_product_s_titles_and_urls(SEARCH_QUERY, TOP_N_PRODUCTS);

<!DOCTYPE html>
<html lang="en">
 <head>
  <link href="https://rukminim2.flixcart.com" rel="preconnect"/>
  <link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.905c37.css" rel="stylesheet"/>
  <link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.d4bb4f.css" rel="stylesheet"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content=
Count of rows(reviews):10



rating:4 


summary:Good quality product 


review:Best phone in this price range 


location:Jaipur 


date:19 days ago 


upvotes:222 


downvotes:47 




Search Results Iteration:   0%|          | 0/24 [00:00<?, ?it/s]

['POCO M6 Pro 5G (Power Black, 128 GB)', 'REDMI 11 Prime (Peppy Purple, 128 GB)']
['https://www.flipkart.com/poco-m6-pro-5g-power-black-128-gb/product-reviews/itmef8fa46f89738?pid=MOBGRNZ3ER4N3K4F&lid=LSTMOBGRNZ3ER4N3K4FIYYGCU&marketplace=FLIPKART&q=phones&store=tyy%2F4io&srno=s_1_1&otracker=search&fm=organic&iid=3a1e44f1-20f6-498e-9e21-2af63a4d197d.MOBGRNZ3ER4N3K4F.SEARCH&ppt=None&ppn=None&ssid=wohptrrzqo0000001693547937104&qH=28388ea49f54c5b8', 'https://www.flipkart.com/redmi-11-prime-peppy-purple-128-gb/product-reviews/itm52d6b0fe396e9?pid=MOBGZHFGUHYSHQYM&lid=LSTMOBGZHFGUHYSHQYMWGOOJ0&marketplace=FLIPKART&q=phones&store=tyy%2F4io&srno=s_1_2&otracker=search&fm=organic&iid=3a1e44f1-20f6-498e-9e21-2af63a4d197d.MOBGZHFGUHYSHQYM.SEARCH&ppt=None&ppn=None&ssid=wohptrrzqo0000001693547937104&qH=28388ea49f54c5b8']


In [1]:
from time import sleep
from random import random
import pandas as pd
import requests
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib.parse as urlparse
from urllib.parse import parse_qs

# Constants
BASE_URL = 'https://www.flipkart.com/'
SEARCH_QUERY = "phones"
TOP_N_PRODUCTS = 20
REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT = 100 #10 Reviews exist per page

SAMPLE_URL = "https://www.flipkart.com/poco-m6-pro-5g-power-black-128-gb/p/itmef8fa46f89738?pid=MOBGRNZ3ER4N3K4F&lid=LSTMOBGRNZ3ER4N3K4FIYYGCU&marketplace=FLIPKART&q=phones&store=tyy%2F4io&spotlightTagId=TrendingId_tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=Search&iid=4b216b86-7d32-47a6-924d-a4b1f3d52950.MOBGRNZ3ER4N3K4F.SEARCH&ppt=sp&ppn=sp&ssid=ygv5uzlqb40000001693306294200&qH=28388ea49f54c5b8"
r = requests.get(SAMPLE_URL)    
soup = BeautifulSoup(r.content, 'html.parser') 
print(soup.prettify()[:500])

# Extracting all review blocks
## Note col._2wzgFH.K0kLPL means 3 entities namely 'col', ' _2wzgFH' and 'K0kLPL' 
## This is written in HTML as 'col _2wzgFH K0kLPL'
## This can also be seen in Bullet 3

rows = soup.find_all('div',attrs={'class':'col _2wzgFH'})
print(f"Count of rows(reviews):{len(rows)}\n\n\n")
# iteration over all blocks
for row in rows:
    # Print a sample row(review html block)
    # print(f"row:\n{row} \n\n")
    
    # finding all rows within the block
    sub_row = row.find_all('div',attrs={'class':'row'})
        
    # extracting text from 1st and 2nd row
    rating = sub_row[0].find('div').text
    print(f"rating:{rating} \n\n")
    
    summary = sub_row[0].find('p').text
    print(f"summary:{summary} \n\n")
    
    review = sub_row[1].find_all('div')[2].text
    print(f"review:{review} \n\n")
    
    location = sub_row[3].find('p',attrs={'class':'_2mcZGG'}).find_all('span')[1].text
    location = "".join(location.split(",")[1:]).strip()
    print(f"location:{location} \n\n")
    
    date = sub_row[3].find_all('p',attrs={'class':'_2sc7ZR'})[1].text
    print(f"date:{date} \n\n")
    
    
    sub_row_2 = row.find_all('div',attrs={'class':'_1e9_Zu'})[0].find_all('span',attrs={'class':'_3c3Px5'})
    
    upvotes = sub_row_2[0].text
    print(f"upvotes:{upvotes} \n\n")
    
    downvotes = sub_row_2[1].text
    print(f"downvotes:{downvotes} \n\n")
    
    break  
  
def get_popular_product_s_titles_and_urls(search_query : str, popular_products_count_limit : int = None):
    
    search_url = f"{BASE_URL}search?q={SEARCH_QUERY}&sort=popularity"
    search_response = requests.get(search_url)
    
    # Pause the loop for 1-3 seconds to simulate natural setting not overwhelm the server with back to back requests without any pause
    #sleep(randint(1,3))
    
    search_html_soup = BeautifulSoup(search_response.content, 'html.parser')
    search_results_products = search_html_soup.find_all('div',attrs={'class':'_2kHMtA'})
    
    product_titles, product_urls,retail_prices,discount_perc,discounted_price = [],[],[],[],[]
    
    product_count = 0
    
    
    for product in tqdm(search_results_products, desc="Search Results Iteration", position=0, leave=True):
        
        ad_mention_subrow = product.find("div", attrs={"class":"_4HTuuX"})
        
        is_ad = not not ad_mention_subrow
        
        if not is_ad:
            
            title_mention_subrow = product.find("a", attrs={"class":"_1fQZEK"})
            product_title=product.find('div', attrs={"class":"_4rR01T"}).text
            product_relative_url = title_mention_subrow["href"]
            Retail_Price= product.find('div', attrs={"class":"_3I9_wc _27UcVY"}).text
            DisPer=product.find('div', attrs={"class":"_3Ay6Sb"}).text
            Disc_price=product.find('div', attrs={"class":"_30jeq3 _1_WHN1"}).text
            #print(product_relative_url)
            product_url = urljoin(BASE_URL,product_relative_url)
            
            parsed_url = urlparse.urlparse(product_url)
            parsed_url_path = parsed_url.path
            parsed_url_path_split = parsed_url_path.split("/")
            parsed_url_path_split[2] = "product-reviews"
            parsed_url_path_modified = "/".join(parsed_url_path_split)
            parsed_url_modified = parsed_url._replace(path=parsed_url_path_modified)
            product_url = parsed_url_modified.geturl()
            
            
            product_titles.append(product_title)
           
            product_urls.append(product_url)
            retail_prices.append(Retail_Price)
            discount_perc.append(DisPer)
            discounted_price.append(Disc_price)
            product_count += 1
            
            if popular_products_count_limit and (product_count >= popular_products_count_limit):
                break
    print(product_titles) 
    print(product_urls)
    return product_titles, product_urls,retail_prices,discount_perc,discounted_price
    
product_titles, product_urls,retail_prices,discount_perc,discounted_price = get_popular_product_s_titles_and_urls(SEARCH_QUERY, TOP_N_PRODUCTS);

<!DOCTYPE html>
<html lang="en">
 <head>
  <link href="https://rukminim2.flixcart.com" rel="preconnect"/>
  <link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.905c37.css" rel="stylesheet"/>
  <link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.d4bb4f.css" rel="stylesheet"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content=
Count of rows(reviews):10



rating:5 


summary:Terrific purchase 


review:a.VFM(Value For Money)b.Best Product of this price.c.If stero speaker was given would have been good.But, this is budget segment we cannot ask too much for this price range.d.Nowadays we ppl are rushing up for going job,school,etc...,18W charger wasn't enough.Atleast they could've given 33W charger.e.I can see there are lot of comments about motherboard issue but, needs to be rectified POCO.f.If you're a passion about 

Search Results Iteration:   0%|          | 0/24 [00:00<?, ?it/s]

['POCO C51 (Royal Blue, 64 GB)', 'POCO C51 (Power Black, 64 GB)', 'REDMI 12 (Moonstone Silver, 128 GB)', 'POCO C50 (Royal Blue, 32 GB)', 'SAMSUNG Galaxy F13 (Sunrise Copper, 64 GB)', 'realme C55 (Rainy Night, 128 GB)', 'realme C55 (Rainforest, 128 GB)', 'realme C55 (Rainforest, 64 GB)', 'SAMSUNG Galaxy F13 (Nightsky Green, 64 GB)', 'realme C55 (Rainy Night, 64 GB)', 'realme C55 (Sunshower, 64 GB)', 'realme C55 (Rainforest, 64 GB)', 'SAMSUNG Galaxy F13 (Waterfall Blue, 64 GB)', 'realme C55 (Sunshower, 128 GB)', 'realme C55 (Sunshower, 64 GB)', 'MOTOROLA e13 (Aurora Green, 64 GB)', 'POCO C50 (Country Green, 32 GB)', 'realme C55 (Rainy Night, 64 GB)', 'POCO C55 (Cool Blue, 128 GB)', 'Nokia 105 Single SIM, Keypad Mobile Phone with Wireless FM Radio']
['https://www.flipkart.com/poco-c51-royal-blue-64-gb/product-reviews/itm1e4e8373537a7?pid=MOBGZCQFCWNDK89P&lid=LSTMOBGZCQFCWNDK89POEPN6S&marketplace=FLIPKART&q=phones&store=tyy%2F4io&srno=s_1_1&otracker=search&fm=organic&iid=11af80a2-442d-42c5

In [3]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["# Products", "# Reviews Per Page", "# Pages", "# Total Reviews Count"]
x.add_row([len(product_urls), 10, REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT, len(product_urls)*10*REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT])
print(x)


dataset = []
#print("After dataset definition")

for idx, url in enumerate(tqdm(product_urls, desc='products')):
    # iterating over review pages
    #print("Inside first for loop")
    for i in tqdm(range(1,REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT+1), desc="review pages", position=0, leave=False):
        parsed = urlparse.urlparse(url)
        pid = parse_qs(parsed.query)['pid'][0]
        URL = f"{url}&page={i}"
        
        r = requests.get(URL)
        
        # Pause the loop for 0-1 seconds to simulate natural setting not overwhelm the server with back to back requests without any pause
        sleep(random())
        soup = BeautifulSoup(r.content, 'html.parser') 

        rows = soup.find_all('div',attrs={'class':'col _2wzgFH K0kLPL'})

        for row in rows:

            # finding all rows within the block
            sub_row = row.find_all('div',attrs={'class':'row'})

            # extracting text from 1st 2nd and 4th row
            rating = sub_row[0].find('div').text
            summary = sub_row[0].find('p').text
            summary = summary.strip()
            review = sub_row[1].find_all('div')[2].text
            review = review.strip()
            location=""
            location_row = sub_row[3].find('p',attrs={'class':'_2mcZGG'})
            if location_row:
                location_row = location_row.find_all('span')
                if len(location_row)>=2:
                    location = location_row[1].text
                    location = "".join(location.split(",")[1:]).strip()
            date = sub_row[3].find_all('p',attrs={'class':'_2sc7ZR'})[1].text

            sub_row_2 = row.find_all('div',attrs={'class':'_1e9_Zu'})[0].find_all('span',attrs={'class':'_3c3Px5'})

            upvotes = sub_row_2[0].text
            downvotes = sub_row_2[1].text
        
            #appending to data
            dataset.append({'product_id':pid, 'product_title':product_titles[idx],'retail_price':retail_prices[idx],'disc_perc':discount_perc[idx],'disc_price':discounted_price[idx],'rating': rating, 'summary': summary, 'review': review, 'location' : location, 'date' : date, 'upvotes' : upvotes, 'downvotes' : downvotes})
print(dataset)

+------------+--------------------+---------+-----------------------+
| # Products | # Reviews Per Page | # Pages | # Total Reviews Count |
+------------+--------------------+---------+-----------------------+
|     20     |         10         |   100   |         20000         |
+------------+--------------------+---------+-----------------------+


products:   0%|          | 0/20 [00:00<?, ?it/s]

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

...

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

review pages:   0%|          | 0/100 [00:00<?, ?it/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
df = pd.DataFrame(dataset)

In [5]:
count_reviews = df.shape[0]
print(f"Count of reviews:{count_reviews}")

Count of reviews:16476


In [6]:
df.to_csv("./flipkart_reviews_datasetfinal.csv", index=False)